# 데이터 전처리의 중요성
데이터 전처리는 원본 데이터를 머신러닝 모델이 더 잘 이해하고 학습할 수 있는 형태로 변환하는 모든 과정을 말한다.<br> 여기에는 스케일링, 인코딩, 결측치 처리 등이 포함된다.

- 왜 중요한가?:<br> 'Garbage In, Garbage Out'이라는 말이 있다. 아무리 좋은 알고리즘을 사용하더라도 데이터의 품질이 낮으면 모델의 성능은 한계에 부딪히게 된다. 데이터 전처리는 모델의 성능을 좌우하는 가장 중요한 단계 중 하나이며, 특히 거리 기반(예: K-NN)이나 경사 하강법 기반(예: 선형 회귀) 알고리즘에서는 필수적이다.


## Scikit-learn의 전처리기(Transformer)
Scikit-learn에서 데이터를 변환하는 역할을 하는 모든 객체들을 통칭다. 이들은 Estimator의 일종으로, 모델(Classifier, Regressor)과 마찬가지로 일관된 `fit()`, `transform()`, `fit_transform()` 인터페이스를 가진다.

모델 학습에 사용되는 Estimator(예: LinearRegression)는 fit()과 predict() 메서드를, 전처리기(예: StandardScaler)는 fit()과 transform() 메서드를 가진다. 이 일관된 구조 덕분에 **Pipeline**을 사용하여 전처리-모델링 과정을 쉽게 결합할 수 있다.

1. 데이터 스케일링(Scaling)<br>
  - 특성들의 값 범위를 동일하게 만들어주는 과정이다. 이는 특정 특성의 값이 다른 특성보다 지나치게 커서 모델 학습에 영향을 미치는 것을 방지한다.
  - StandardScaler: 데이터를 평균이 0, 표준편차가 1인 표준 정규 분포 형태로 변환한다. 이상치(Outlier)에 민감할 수 있지만, 대부분의 경우 가장 널리 사용된다.
  $$z = \frac{x - \mu}{\sigma}$$

  - MinMaxScaler: 데이터를 0과 1 사이의 값으로 변환한다. 이상치에 매우 민감하여 데이터에 이상치가 많을 경우 성능 저하를 일으킬 수 있다.
  $$X_{\text{norm}} = \frac{X - X_{\text{min}}}{X_{\text{max}} - X_{\text{min}}}$$
  - RobustScaler: 중앙값(median)과 사분위 범위(IQR)를 사용하여 데이터를 변환합니다. 이상치의 영향을 최소화해야 할 때 유용하다.

2. 인코딩(Encoding)
  - 남자, 여자, 서울, 부산과 같은 텍스트(범주형) 데이터를 머신러닝 모델이 이해할 수 있는 숫자 형태로 변환하는 과정이다.
  - LabelEncoder: 1차원 배열의 범주형 데이터를 순서대로 0부터 N-1까지의 정수형 숫자로 변환한다.
    - 한계점:<br> 사과:0, 바나나:1, 딸기:2와 같이 변환하면 모델이 이 숫자들 사이에 사과 < 바나나 < 딸기와 같은 순서 관계가 있다고 오해할 수 있다. 따라서 순서가 없는 범주형 데이터에는 사용하지 않는 것이 좋다.
  - OneHotEncoder: 범주형 데이터를 이진(0 또는 1) 벡터로 변환한다. <br>예를 들어, 서울, 부산, 제주는 [1, 0, 0], [0, 1, 0], [0, 0, 1]로 변환된다.
    - 장점: 숫자 간의 순서 관계가 발생하지 않아 모델이 데이터를 왜곡하여 해석하는 문제를 해결할 수 있다.
    - 주의사항: 특성의 개수가 매우 많아지면 메모리 낭비가 심해질 수 있다. 이럴 경우 sparse=True 옵션을 활용하여 효율적으로 관리할 수 있다.


3. 결측치 처리(Imputation)
  - 데이터에 비어있는 값(NaN, None)을 적절한 값으로 채워 넣는 과정이다.
  - SimpleImputer: 결측치를 특정 전략(strategy)으로 채운다.
    - strategy: 'mean' (평균), 'median' (중앙값), 'most_frequent' (최빈값), 'constant' (고정된 값) 등의 옵션을 제공한다.



## 예제 : 데이터 스케일링
`StandardScaler`를 사용하여 데이터를 스케일링하고, 원본 데이터와 변환된 데이터의 분포를 비교한다.

1. 데이터 생성 및 준비:<br> 길이가 다른 두 개의 특성(Feature 1, Feature 2)을 가진 가상의 데이터셋을 생성

In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# 가상의 데이터셋 생성
# Feature 1은 값이 작고, Feature 2는 값이 큰 특성
X = np.array([[10, 1000], [20, 2000], [30, 3000]])
print("원본 데이터 X:\n", X)

원본 데이터 X:
 [[  10 1000]
 [  20 2000]
 [  30 3000]]


2. `StandardScaler` 적용: <br>fit_transform() 메서드를 사용하여 데이터를 스케일링한다.

In [4]:
# StandardScaler 객체 생성
scaler = StandardScaler()

# 데이터 스케일링
X_scaled = scaler.fit_transform(X)

print("\n스케일링된 데이터 X_scaled:\n", X_scaled)

print("\n스케일링 후 평균:", X_scaled.mean(axis=0))
print("스케일링 후 표준편차:", X_scaled.std(axis=0))
# 축(axis)을 지정하는 옵션 0 : 열, 1 : 행


스케일링된 데이터 X_scaled:
 [[-1.22474487 -1.22474487]
 [ 0.          0.        ]
 [ 1.22474487  1.22474487]]

스케일링 후 평균: [0. 0.]
스케일링 후 표준편차: [1. 1.]


> 결론<br>
각 열을 별개로 계산하기 때문에, 원본 데이터의 값이 다르더라도 계산 과정과 결과는 비례적으로 동일하게 나온다.<br>
이러한 스케일링을 통해 모든 특성은 평균이 0이고 표준편차가 1인 새로운 분포를 가지게 된다.
이는 '나이'와 '연봉'의 스케일 차이로 인해 한쪽 특성(연봉)에만 과도한 가중치가 부여되는 것을 방지하고, 모델이 모든 특성을 공평하게 학습할 수 있도록 돕는다.

## 범주형 데이터 인코딩
LabelEncoder와 OneHotEncoder의 차이를 이해하고, 데이터의 특성에 맞게 올바른 인코딩 방법을 선택하는 방법을 학습한다.

### LabelEncoder
LabelEncoder는 문자열로 된 범주형 데이터를 0부터 시작하는 정수 레이블로 변환한다

1. 데이터 준비: 순서가 없는 범주형 데이터(색상)와 순서가 있는 범주형 데이터(등급)를 준비.

In [5]:
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# 가상의 범주형 데이터
colors = ['Red', 'Green', 'Blue', 'Red', 'Blue']
grades = ['C', 'A', 'B', 'A', 'C']

2. LabelEncoder 적용: 순서가 있는 데이터에만 LabelEncoder를 적용하고 결과를 확인.

In [6]:
# LabelEncoder는 1차원 배열에만 적용 가능
le = LabelEncoder()
grades_encoded = le.fit_transform(grades)

print("원본 등급:", grades)
print("LabelEncoder 적용 후 등급:", grades_encoded)
print("변환된 클래스:", le.classes_)

원본 등급: ['C', 'A', 'B', 'A', 'C']
LabelEncoder 적용 후 등급: [2 0 1 0 2]
변환된 클래스: ['A' 'B' 'C']


### OneHotEncoder
One-Hot Encoding은 범주형 데이터를 **이진 벡터(Binary Vector)**로 변환하는 기법이다


In [7]:
# OneHotEncoder는 2차원 배열에 적용
colors_2d = np.array(colors).reshape(-1, 1)
# np.array : 판다스(Pandas) 데이터프레임이나 넘파이(NumPy) 배열 같은 배열 형태의 데이터를 입력으로 사용
# reshape(-1, 1) : 각 샘플(행)에 하나의 특성(열)이 있는 2차원 배열을 입력으로 기대하기 때문
# -1 : 자동으로 행의 수를 계산, 1은 열의 수가 1개

ohe = OneHotEncoder(sparse_output=False) # sparse_output=False로 NumPy 배열 출력
colors_onehot = ohe.fit_transform(colors_2d)

print("\nOneHotEncoder 적용 전 색상:\n", colors_2d)
print("\nOneHotEncoder 적용 후 색상:\n", colors_onehot)


OneHotEncoder 적용 전 색상:
 [['Red']
 ['Green']
 ['Blue']
 ['Red']
 ['Blue']]

OneHotEncoder 적용 후 색상:
 [[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
